https://python.langchain.com/docs/tutorials/agents/#setup

In [ ]:
%pip install -U langchain-community langgraph langchain_openai tavily-python langgraph-checkpoint-sqlite

In [8]:
from dotenv import load_dotenv
import os
from langchain_community.tools.tavily_search import TavilySearchResults

In [3]:
load_dotenv()

TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

https://python.langchain.com/docs/integrations/chat/openai/

In [ ]:
search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

In [5]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4o",
    api_key=OPENAI_API_KEY
)

In [ ]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="フェルマーの最終定理について解説して")])
response.content

We can now see what it is like to enable this model to do tool calling. In order to enable that we use .bind_tools to give the language model knowledge of these tools

このモデルでツールを呼び出せるようにするにはどうすればいいか、見てみよう。 それを可能にするために、.bind_toolsを使い、言語モデルにこれらのツールに関する知識を与えている

In [10]:
model_with_tools = model.bind_tools(tools)

In [ ]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

In [ ]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

### エージェントの作成 

ツールとLLMを定義したので、エージェントを作成します。 
エージェントの作成にはLangGraphを使います。 
現在のところ、エージェントを構築するために高レベルのインタフェースを使用していますが、
LangGraphの良いところは、この高レベルのインタフェースが、エージェントのロジックを変更したい場合に備えて、
低レベルで高度に制御可能なAPIに支えられていることです。 

さて、LLMとツールでエージェントを初期化しましょう。 
model_with_toolsではなく、modelを渡していることに注意してください。 
これは、create_react_agent が .bind_tools を呼び出すからです。

In [13]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

### エージェントの実行 

いくつかのクエリでエージェントを実行することができます！ 
今のところ、これらはすべてステートレスクエリであることに注意してください（以前のやりとりを覚えていません）。 
エージェントはインタラクションの最後に最終的な状態を返すことに注意してください
（これはあらゆる入力を含みますが、出力だけを取得する方法は後で説明します）。 

まず最初に、ツールを呼び出す必要がないときにどのように応答するかを見てみましょう：

In [ ]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

In [ ]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

メッセージのストリーミング 

エージェントを.invokeで呼び出して、最終的なレスポンスを返す方法について見てきました。 
エージェントが複数のステップを実行する場合、時間がかかるかもしれません。 
途中経過を表示するために、発生したメッセージをストリームバックすることができます。

In [ ]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="株式会社hogeの評判について教えてください")]}
):
    print(chunk)
    print("----")

In [ ]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="株式会社hogeの評判について教えてください")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

メモリーへの追加 

前述のように、このエージェントはステートレスである。 
つまり、以前のやりとりを覚えていない。 
記憶させるためには、チェックポインタを渡す必要がある。 
チェックポインタを渡すとき、
エージェントを起動するときにスレッドIDも渡す必要があります
（どのスレッド/会話から再開するかを知るため）。

In [21]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [22]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [ ]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im naoki!")]}, config
):
    print(chunk)
    print("----")

In [ ]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

まとめ 

以上です！ 
このクイックスタートでは、シンプルなエージェントの作成方法を説明しました。 

そして、中間ステップだけでなく、トークンも含めてレスポンスをストリームバックする方法を示しました！ 

さらに、エージェントと会話ができるようにメモリも追加しました。 
エージェントは複雑なトピックで、学ぶことがたくさんあります！
エージェントに関する詳しい情報は、LangGraphのドキュメントをご覧ください。 

これには独自のコンセプト、チュートリアル、ハウツーガイドがあります。